In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os

In [2]:
def read_vocab(filename):
    
    dict_ = {}
    with open(filename, 'r', encoding='utf-8-sig') as datafile:
        for line in datafile:
            occurence = line.strip().split()[0]
            word = line.strip().split()[1]
            dict_[word] = int(occurence)
    return dict_

In [3]:
import os
os.chdir(r'D:/Documents/etudes/epfl/MA1/cours/MachineLearning/Project2/data/twitter_datasets_epfl/full/')

In [ ]:
#  vocab pos
os.system('build_vocab_pos_full.sh')
os.system('cut_vocab_pos_full.sh')

#  vocab neg
os.system('build_vocab_neg_full.sh')
os.system('cut_vocab_neg_full.sh')

In [4]:
# read the vocabs
vocab_neg = read_vocab('vocab_neg_full.txt')
vocab_pos = read_vocab('vocab_pos_full.txt')

In [5]:
import operator

def extract_most_freq(vocab_dict, max_words=10):
    
    most_freq_words = []
    vocab_list_sorted = sorted(vocab_dict.copy().items(), key=operator.itemgetter(1), reverse = True)

    for i in range(max_words):
        most_freq_words.append((vocab_list_sorted[i][0], vocab_list_sorted[i][1]))
    
    return most_freq_words

In [6]:
most_neg = extract_most_freq(vocab_neg, max_words=10)
print("\n".join("{}\t{}".format(x[0], x[1]) for x in most_neg))

.	751865
i	625066
<user>	557268
(	461668
<number>	420633
<repeat>	414073
the	384681
<url>	359419
,	357807
!	356189


In [7]:
most_pos = extract_most_freq(vocab_pos, max_words=10)
print("\n".join("{}\t{}".format(x[0], x[1]) for x in most_pos))

﻿<user>	951004
!	585557
i	543778
.	499484
you	386420
to	317352
,	305792
the	297102
a	239967
is	239662


In [8]:
def word_ratio_neg_pos(vocab_dict_neg, vocab_dict_pos):
    
    word_ratio = {}

    # scan negative tweets
    for word_neg, occu_neg in vocab_dict_neg.items():  
        
        # if word in both neg and pos vocab
        if word_neg in vocab_dict_pos:
            
            # get occurence in positive tweets
            occu_pos = vocab_dict_pos.get(word_neg)
            
            # compute the ratio
            word_ratio[word_neg] = [occu_neg/occu_pos, occu_neg, occu_pos]
            
    return word_ratio

In [9]:
freq_min = 1
vocab_neg_thr = {k: v for k, v in vocab_neg.copy().items() if v > freq_min}
vocab_pos_thr = {k: v for k, v in vocab_pos.copy().items() if v > freq_min}

In [10]:
word_ratio = word_ratio_neg_pos(vocab_neg_thr, vocab_pos_thr)

In [11]:
print(word_ratio.get('happy'))
print(word_ratio.get('not'))

[0.2264985696227796, 6809, 30062]
[1.5165902801165565, 207145, 136586]


In [18]:
# build exclusion vocab with stop words
ratio_margin = 0.1
freq_min = 100
stop_vocab = {}

for word, values in word_ratio.items():
    
    ratio = values[0]
    occu_neg = values[1]
    occu_pos = values[2]
    
    # remove non discriminant word
    if ratio > 1 - ratio_margin and ratio < 1 + ratio_margin:
        stop_vocab[word] = values
        
    # remove unfrequent word
    if occu_neg < freq_min and occu_pos < freq_min:
        stop_vocab[word] = values
    
print("Number of stop words: ", len(stop_vocab))

Number of stop words:  47290


In [19]:
print("\n".join("{}\t{}".format(k,v) for k,v in stop_vocab.items()))

	[0.6666666666666666, 2, 3]
'	[1.040695000985351, 31685, 30446]
(<number>)	[2.2, 11, 5]
(<number><smile>	[1.0, 2, 2]
(=	[1.0588235294117647, 36, 34]
*	[0.9180648876943751, 23911, 26045]
:'	[4.25, 68, 16]
:;	[1.3333333333333333, 8, 6]
:-@	[2.0, 4, 2]
:[	[2.090909090909091, 46, 22]
:-[	[4.5, 9, 2]
:-\	[2.5, 40, 16]
:']	[1.5, 6, 4]
:-]	[0.5, 3, 6]
:{	[1.2692307692307692, 33, 26]
:}	[0.6515151515151515, 43, 66]
:'}	[1.25, 5, 4]
:-}	[0.4, 2, 5]
:<	[2.0, 8, 4]
:<number>	[3.4285714285714284, 24, 7]
:=	[0.5, 4, 8]
:o(	[4.0, 8, 2]
:o)	[0.37037037037037035, 10, 27]
:o:	[0.7368421052631579, 14, 19]
:op	[0.2727272727272727, 3, 11]
;-	[1.5, 3, 2]
;:	[3.9, 39, 10]
;\	[1.0, 2, 2]
;]	[0.5813953488372093, 25, 43]
;}	[0.6666666666666666, 6, 9]
;o)	[0.42857142857142855, 3, 7]
[:	[0.46153846153846156, 18, 39]
[;	[0.5, 3, 6]
[=	[1.0, 3, 3]
\	[0.9848208864602307, 1622, 1647]
\:	[0.4444444444444444, 8, 18]
\=	[0.4, 8, 20]
_	[0.9908675799086758, 5208, 5256]
__	[0.9632107023411371, 864, 897]
_<number>	[0.625,

In [ ]:
import os
os.chdir(r'D:/Documents/etudes/epfl/MA1/cours/MachineLearning/Project2/data/stop_words/')

In [ ]:
# write stop word in a text
outputfilename = "stop_word_" + "freq_min_" + str(freq_min) + "_ratio_marg_" + str(ratio_margin) + ".txt"
with open(outputfilename, 'w', encoding='utf-8-sig') as outfile:
    for word, value in stop_vocab.items():
        outfile.write(word)   
        outfile.write("\n")      
outfile.close()